# Overview
LitReview now uses an Object-Graph Model (OGM) called `neomodel` which is used to generate python types from Neo4j queries. There are many benefits of using an OGM:
* Type Safety: graph query responses are strongly typed, which makes it easier to read the code and detect errors
* Complexity Reduction: OGMs simplify common behavior by wrapping queries and commands in functions
* Explicit Modeling: Defining the types in code means the database can be built from the models directly
* Asynchronous: Using the `adb` driver, queries can be executed asynchronously  


# Usage
## Connecting

In [14]:
from db.util import neomodel_connect
res = await neomodel_connect()
print(res.message)

Successfully connected to Neo4j!


## Querying
### Cypher with Pandas

In [15]:
# Using cypher and pandas
from neomodel.integration.pandas import to_dataframe
from neomodel import adb

res = await adb.cypher_query("MATCH (a:Author) RETURN a.name as Name, a.author_id as author_id")
df = to_dataframe(res)
df.head()

,Name,author_id
0,Kevin R. Moon,None
1,David van Dijk,None
2,Z. Wang,None
3,Scott A. Gigante,None
4,Daniel B. Burkhardt,None


### Neomodel

In [16]:
from db.models import Author, Paper

# Query all author and paper nodes
people = await Author.nodes.all()
papers = await Paper.nodes.all()

In [17]:
# Add a new author and paper
author = Author(name="John Doe", author_id="1234")
paper = Paper(title="My Paper", paper_id="5678", abstract="This is my paper")

# Save
await author.save()
await paper.save()

<Paper: {'paper_id': '5678', 'title': 'My Paper', 'abstract': 'This is my paper', 'year': None, 'reference_count': None, 'citation_count': None, 'influential_citation_count': None, 'publication_types': [], 'publication_date': None, 'element_id_property': '4:a80c54f2-e783-4768-86e7-1415a1b5328e:742'}>

In [18]:
# Add relationship
await author.papers.connect(paper)

True

In [19]:
# Query author by name
queried_author = await Author.nodes.filter(name="John Doe").first_or_none()
if queried_author:
    print(f"Found author: {queried_author.name}")

Found author: John Doe


In [20]:
# Update paper details
paper.abstract = "Updated abstract content"
await paper.save()
print(f"Updated paper abstract: {paper.abstract}")

Updated paper abstract: Updated abstract content


In [21]:
# Retrieve all papers authored by this author
authored_papers = await author.papers.all()

# Retrieve all authors of this paper
paper_authors = await paper.authors.all()

print(f"{author.name} has authored {len(authored_papers)} papers.")
print(f"{paper.title} has {len(paper_authors)} authors.")

John Doe has authored 1 papers.
My Paper has 1 authors.


In [22]:
# Disconnect relationship
await author.papers.disconnect(paper)
print(f"Disconnected {author.name} from {paper.title}")

Disconnected John Doe from My Paper


In [23]:
# Delete nodes
await author.delete()
await paper.delete()
print(f"Deleted {author.name} and {paper.title}")

Deleted John Doe and My Paper
